In [40]:
# 使用bi-lstm作为基本结构，进行模型预测，采用pytorch实现
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import time 
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SOS_token = 0
EOS_token = 1

MAX_LENGTH = 10

def asMinutes(s):
    m = math.floor(s / 60) 
    s -= m * 60 
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2) 
    ax.yaxis.set_major_locator(loc) 
    plt.plot(points)


In [41]:
# data preprocess
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {0:'SOS',1:'EOS'}
        self.word2count = {}
        self.n_words = 2 # include SOS and EOS
        
    def addSentence(self, sentence):
        for item in sentence.split(' '):
            self.addWord(item)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
import unicodedata
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


# Lowercase, trim, and remove non-letter characters
import re
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs


# filter pairs to english sentence start with eng_prefixes
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


# prepareData 
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

import random
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['nous ne nous y rendons pas .', 'we re not going .']


In [42]:
# 这是一个单层单向的encoder GRU单元的实现 
class EncoderGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(input_size = hidden_size, hidden_size = hidden_size)
    
    def forward(self, x, hidden): 
        # x shape: [1] ? 
        # embedded shape: [1,1, hidden_size]
        embedded = self.embedding(x).view(1,1,-1)        # 这里的1是batch_size，1是输入序列的长度，-1是输入序列的维度
        output = embedded
        # hidden shape: [1, 1, hidden_size] for [D∗num_layers, batch_size, hidden_size] 
        # output shape: [1, 1, hidden_size] for [batch_size, sequence_length, hidden_size]
        output, hidden = self.gru(output, hidden)
        return output, hidden 
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
# 这是一个单层单向的decoder GRU单元的实现 
class DecoderGRU(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderGRU,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size) 
        self.gru = nn.GRU(input_size = hidden_size, hidden_size = hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        # input shape: [1]
        # hidden shape: [1, 1, hidden_size] 
        # output shape: [1, 1, hidden_size]        
        output = self.embedding(input).view(1, 1, -1)
        # element-wise relu
        output = F.relu(output)
        # output shape: [1, 1, hidden_size]
        output, hidden = self.gru(output, hidden)
        # output[0] shape: [1, hidden_size]
        # self.out(output[0]) shape: [1, output_size] 
        # self.softmax(self.out(output[0])) shape: [output_size]
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [43]:
# 这是加入了attention机制的一个单层单向的decoder GRU单元的实现 
# Luong Attention 
class AttDecoderGRU(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(AttDecoderGRU,self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size 
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(input_size = hidden_size, hidden_size = hidden_size)
        # used to compute Ht_hat from concated context vector and current hidden state with Wc
        self.linear = nn.Linear(hidden_size*2, hidden_size) 
        # compute output with Ws
        self.out = nn.Linear(hidden_size, output_size) 
        
    def forward(self, input, hidden, encoder_outputs):
        # encoder_outputs shape: [1, max_len, hidden_size] 
        # input shape: [1]
        # embedded shape: [1,1, hidden_size]
        embedded = self.embedding(input).view(1, 1, -1)
        
        # compute gru  
        # output shape: [1, 1, hidden_size] for [batch_size, sequence_length, hidden_size]
        # hidden shape: [1, 1, hidden_size] for [D∗num_layers, batch_size, hidden_size] 
        output, hidden = self.gru(embedded, hidden)
        
        # attention weights = encoder_outputs * output
        # attn_weights shape: [1, max_len, 1]
        attn_weights = F.softmax(torch.bmm(encoder_outputs, output.transpose(1,2)), dim = 1)
        # context vector shape: [1, 1, hidden_size]
        context_vector = torch.bmm(attn_weights.transpose(1,2), encoder_outputs)
        
        # ht_hat shape: [1,1,hidden_size]
        # compute element-wise tanh
        ht_hat = F.tanh(self.linear(torch.cat((context_vector, hidden), dim = 2)))
        
        # output loglikelihood softmax result
        # output shape: [1, 1, output_size]
        # hidden shape: [1, 1, hidden_size]
        output = F.log_softmax(self.out(ht_hat), dim = 2) 
        
        return output, hidden
    
    def initHidden(self): 
        return torch.zeros(1, 1, self.hidden_size, device=device) 

In [44]:
teacher_forcing_ratio = 0.5

# one step training with one sample at a time (SGD)
def train(input_tensor, 
          target_tensor, 
          encoder, decoder, 
          encoder_optimizer, 
          decoder_optimizer, 
          loss_function, 
          max_length=MAX_LENGTH):
    
    # initialize input hidden state, shape:[1,1,hidden_size]
    encoder_hidden = encoder.initHidden()
    
    # every step recompute gradient, reset gradient to zeros
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    # encoder_outputs shape: [1, max_length, encoder.hidden_size]
    encoder_outputs = torch.zeros(1, max_length, encoder.hidden_size, device=device) 
    
    loss = 0
    
    # encoder
    for index in range(input_length):
        # encoder_output shape: [1, 1, encoder.hidden_size]
        encoder_output, encoder_hidden = encoder.forward(input_tensor[index], encoder_hidden)
        encoder_outputs[0,index] = encoder_output[0,0] 
        
    
    # decoder
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            # decoder_output shape: [1, 1, output_size]
            decoder_output, decoder_hidden = decoder.forward(decoder_input, decoder_hidden, encoder_outputs)
            loss += loss_function(decoder_output[0,0], target_tensor[di,0])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder.forward(decoder_input, decoder_hidden, encoder_outputs)
            # topi shape: [1,1,1], topk return the top k largest value in dimension -1
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += loss_function(decoder_output[0,0], target_tensor[di,0])
            if decoder_input.item() == EOS_token:
                break
    
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length

In [51]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

import torch.optim as optim

# training entrance 
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    loss_function = nn.NLLLoss()
    
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_function)
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print("%s, (%d, %d%%), %.4f" % (timeSince(start, iter/n_iters), iter, iter/n_iters*100, print_loss_avg))

    
    showPlot(plot_losses)
        

In [60]:
def evaluate(encoder, decoder, input_sentence, max_length=MAX_LENGTH):
    input_tensor = tensorFromSentence(input_lang, input_sentence)
    input_length = input_tensor.size(0)
    encoder_hidden = encoder.initHidden()
    
    # ??? 这里为什么不直接使用input_sentence的长度，而是使用max_length呢？
    encoder_outputs = torch.zeros(1, max_length, encoder.hidden_size, device=device)
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder.forward(input_tensor[ei], encoder_hidden)
        encoder_outputs[0,ei] = encoder_output[0,0]
        
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    
    decoded_words = []
    
    for di in range(max_length):
        decoder_output, decoder_hidden = decoder.forward(decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        # ??? topi.squeeze().item() ???
        if topi.squeeze().item() == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[topi.squeeze().item()])
        
        decoder_input = topi.squeeze().detach()
        
    return decoded_words

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [52]:
# start training 
hidden_size = 256

encoder1 = EncoderGRU(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttDecoderGRU(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=1000)

0m 27s (- 33m 38s), (1000, 1%), 3.5482
0m 55s (- 33m 56s), (2000, 2%), 3.0806
1m 23s (- 33m 14s), (3000, 4%), 2.7069
1m 47s (- 31m 41s), (4000, 5%), 2.5957
2m 13s (- 31m 6s), (5000, 6%), 2.4887
2m 42s (- 31m 12s), (6000, 8%), 2.4213
3m 14s (- 31m 31s), (7000, 9%), 2.2865
3m 41s (- 30m 52s), (8000, 10%), 2.2036
4m 9s (- 30m 28s), (9000, 12%), 2.1520
4m 40s (- 30m 20s), (10000, 13%), 2.1080
5m 11s (- 30m 10s), (11000, 14%), 2.0323
5m 39s (- 29m 43s), (12000, 16%), 1.9124
6m 7s (- 29m 14s), (13000, 17%), 1.8526
6m 34s (- 28m 39s), (14000, 18%), 1.8173
7m 0s (- 28m 1s), (15000, 20%), 1.8149
7m 26s (- 27m 26s), (16000, 21%), 1.7030
7m 51s (- 26m 48s), (17000, 22%), 1.7105
8m 15s (- 26m 10s), (18000, 24%), 1.6265
8m 42s (- 25m 39s), (19000, 25%), 1.5216
9m 11s (- 25m 16s), (20000, 26%), 1.5650
9m 39s (- 24m 49s), (21000, 28%), 1.4821
10m 6s (- 24m 22s), (22000, 29%), 1.5157
10m 37s (- 24m 0s), (23000, 30%), 1.4529
11m 9s (- 23m 42s), (24000, 32%), 1.3489
11m 36s (- 23m 12s), (25000, 33%), 1.

In [62]:
evaluateRandomly(encoder1, attn_decoder1)

> tu es idiote .
= you re being silly .
< you re being silly . <EOS>

> je ne suis pas ta servante .
= i m not your servant .
< i m not your servant . <EOS>

> nous n y sommes pas habitues .
= we re not used to it .
< we re not used to it . <EOS>

> tu es pire que tom .
= you re worse than tom .
< you re worse than tom . <EOS>

> tu es triste .
= you re sad .
< you re sad . <EOS>

> c est ma fille .
= she s my daughter .
< she s my daughter . <EOS>

> il a peur des serpents .
= he s afraid of snakes .
< he s afraid of snakes . <EOS>

> je ne vais pas m impliquer .
= i am not getting involved .
< i m not going . . <EOS>

> tu as parfaitement raison .
= you re perfectly right .
< you are absolutely right . <EOS>

> tu n aides pas tom .
= you re not helping tom .
< you re not helping tom . <EOS>

